In [1]:
from  datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plot 
import random
import lightgbm as lgb

/Users/hshan/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


read_load
melt_merge
split_data
missing val
encoding cat var
lag by 7, 28
selected col
/Users/hshan/Downloads/M5/sell_prices.csv


In [58]:
def load_df():
    
    cal_cat_cols = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    sales_cat_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    price_cat_cols = ['store_id', 'item_id']
    
    calendars = pd.read_csv('/Users/hshan/Downloads/M5/calendar.csv')
    sales = pd.read_csv('/Users/hshan/Downloads/M5/sales_train_validation.csv')
    prices = pd.read_csv('/Users/hshan/Downloads/M5/sell_prices.csv')
    
    ind_var = ['id'] + sales_cat_cols
    val_var = [col for col in sales.columns if col.startswith('d_')]
    df = pd.melt(sales, id_vars = ind_var, value_vars = val_var, var_name = 'd', value_name='sales')
    df = df.merge(calendars, on = 'd', copy = False)
    df = df.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    cat_cols = cal_cat_cols + sales_cat_cols
    label_encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = df[col].fillna('').astype('category')
        df[col] = label_encoder.fit_transform(df[col])

    unused_cols = ['date','wm_yr_wk', 'weekday']
    
    df.drop(unused_cols, inplace = True, axis = 1)
    
    return (df)


In [3]:
def lag_features(df):
    '''max lag should not be exceeding 57 in this case'''
    num = [1,7,28]
    lags = num
    windows = num
    lag_cols = [f'lag_{lag}' for lag in lags]
    

    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[['id','sales']].groupby('id')['sales'].shift(lag)
        
    for window in windows:
        for lag, lag_col in zip(lags, lag_cols):
            mean_col = f'mean_{lag}_{window}'
            df[mean_col] = df[['id',lag_col]].groupby('id')[lag_col].transform(lambda x: x.rolling(window).mean())
    

    return (df)


In [4]:
categorical_cols = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'] + \
    ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
removed_cols = ['id', 'date', 'sales', 'd', 'wm_yr_wk', 'weekday']

In [5]:
def pca(x_features_df, n_components):
    pca = PCA(n_components)
    pca_cols = []
    for i in range(n_components):
        pca_col = f'pc_{i+1}'
        pca_cols.append(pca_col)
    p_components = pca.fit_transform(x_features_df)
    pca_features_df = pd.DataFrame(data = p_components, columns = pca_cols)
    
    return(pca_features_df)

In [6]:
pred_period = 28
train_start = 1
train_last = 1913
test_start = train_last + 1
test_last  = train_last + pred_period
pred_start = test_last + 1
pred_last = test_last + pred_period
seed = 123

In [7]:
df = load_df()
df = lag_features(df)
df.dropna(inplace=True)

In [8]:
df_cols = list(df.columns)
x_features =[]
for feature in df_cols:
    if feature not in removed_cols:
        x_features.append(feature)
y_target = 'sales'

In [9]:
n_components = int(0.65*(len(df_cols)-2))

In [10]:
pca_features_df = pca(df[x_features], n_components)
y_target = df['sales']

In [11]:
xTrain, xVal, yTrain, yVal = train_test_split(pca_features_df, y_target, test_size = 0.3, random_state = seed)

del df

In [12]:
del pca_features_df, y_target

In [13]:
lgbm_model = lgb.LGBMModel(
    objective = 'poisson',
    boosting_type = 'gbdt',
    metric ='rmse',
    force_row_wise = False,
    learning_rate = 0.075,
    lambda_l2 = 0.1,
    verbosity = 1,
    num_iterations = 500,
    num_leaves = 100,
    boost_from_average = False,
    random_state = seed,
    n_estimators = 300,
    )

gridParams = {
    'learning_rate': 0.01,
    'lambda_l2' : 0.2,
    'num_leaves': 200,
    'num_iterations' : 200,
    'n_estimators': [100]
    }

grid = GridSearchCV(lgbm_model, gridParams, cv=3, n_jobs=-1, scoring="accuracy")

ValueError: Parameter values for parameter (learning_rate) need to be a sequence(but not a string) or np.ndarray.

In [20]:
param = {
    'objective' : 'poisson',
    'boosting_type' : 'gbdt',
    'metric' : 'rmse',
    'force_row_wise' : True,
    'learning_rate' : 0.075,
    'lambda_l2' : 0.1,
    'verbosity' : 1,
    'num_iterations' : 1500,
    'num_leaves' : 150,
    'boost_from_average' : False,
    'random_state' : seed,
    'n_estimators' : 300,
}

In [41]:
param = {
    "objective" : "poisson",
    "metric" :"rmse",
    'boosting_type' : 'gbdt',
    "force_row_wise" : True,
    "learning_rate" : 0.075,
    #"sub_feature" : 0.8,
    "sub_row" : 0.75,
    "bagging_freq" : 1,
    "lambda_l2" : 0.1,
    "nthread" : 5,
    "metric": "rmse",
    'verbosity': 1,
    'num_iterations' : 3000,
    'num_leaves': 250,
    "min_data_in_leaf": 200,
}

In [15]:
lgbm_train_data = lgb.Dataset(xTrain, label = yTrain)
lgbm_valid_data = lgb.Dataset(xVal, label = yVal)

In [16]:
del xTrain, yTrain, xVal, yVal

In [42]:
model = lgb.train(param, train_set=lgbm_train_data, valid_sets=lgbm_valid_data, verbose_eval=100,
                  early_stopping_rounds=40)

Training until validation scores don't improve for 40 rounds
[100]	valid_0's rmse: 2.42532
[200]	valid_0's rmse: 2.39154
[300]	valid_0's rmse: 2.37301
[400]	valid_0's rmse: 2.36242
[500]	valid_0's rmse: 2.35494
[600]	valid_0's rmse: 2.34935
[700]	valid_0's rmse: 2.34614
[800]	valid_0's rmse: 2.34344
[900]	valid_0's rmse: 2.34099
[1000]	valid_0's rmse: 2.33874
[1100]	valid_0's rmse: 2.33776
[1200]	valid_0's rmse: 2.33625
[1300]	valid_0's rmse: 2.33498
[1400]	valid_0's rmse: 2.33424
[1500]	valid_0's rmse: 2.33339
[1600]	valid_0's rmse: 2.33286
[1700]	valid_0's rmse: 2.33241
[1800]	valid_0's rmse: 2.33171
[1900]	valid_0's rmse: 2.3312
[2000]	valid_0's rmse: 2.33068
[2100]	valid_0's rmse: 2.33008
Early stopping, best iteration is:
[2138]	valid_0's rmse: 2.32998


In [39]:
del model

In [ ]:
grid.fit(pca_features_df, y_target)

In [ ]:
params['learning_rate'] = grid.best_params_['learning_rate']
params['num_leaves'] = grid.best_params_['num_leaves']
params['lambda_l2'] = grid.best_params_['lambda_l2']
params['num_iterations'] = grid.best_params_['num_iterations']
params['subsample'] = grid.best_params_['subsample']
params['n_estimators'] = grid.best_params_['n_estimators']


In [43]:
del lgbm_train_data, lgbm_valid_data

In [51]:
pred_df = load_df()
pred_df = lag_features(pred_df)

In [56]:
import joblib

In [ ]:
filename = 'model.sav'
joblib.dump(model, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)

In [50]:
pred_df['sales'].fillna(0, inplace=True)
pred_df = pred_df.loc[test_start:pred_last]
del pred_df

In [ ]:
pred_x_pca_df = pca(pred_df[x_features, n_components)

pred_df['sales'] = model.predict(pred_x_pca_df)
result = pred_df[['id', 'd', 'sales']]

def submission_df():
    df = load_data()
    submission['id'] = df['id']
    del df 
    submission = pd.DataFrame()
    for i in range(pred_period):
        submission_col = f'd_{i}'
        submission[submission_col] = pd.Series()
        
    return (submission_df)

df_unmelted = result.pivot(index='id', columns='d')
df_unmelted = df_unmelted['Value'].reset_index()
df_unmelted.columns.name = None

submission_df.merge(df_unmelted, on = 'id')
for i in range(pred_period):
    col = f'F{i+1}'
    ori = f'd_{i+1}'
    submission_df.rename(columns = {ori:col})

submission_df.to_csv('/Users/hshan/Downloads/M5/submission.csv')

In [ ]:
X_test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state = 12)
    
del X, y; gc.collect();

d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid) 

watchlist = [d_train, d_valid]


model = lgb.train(params, train_set=d_train, num_boost_round=1000, valid_sets=watchlist, early_stopping_rounds=50, verbose_eval=4)

p_test = model.predict(X_test)

subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
submName = strftime("%Y%m%d%H%M%S", gmtime()) + '_submission.csv.gz'
subm.to_csv(submName, compression = 'gzip', index=False, float_format = '%.5f')

In [ ]:
cal_dtype={
    "event_name_1": "category", 
    "event_name_2": "category", 
    "event_type_1": "category", 
    "event_type_2": "category", 
    "weekday": "category", 
    'wm_yr_wk': 'int16', 
    "wday": "int16",
    "month": "int16", 
    "year": "int16", 
    "snap_CA": "float32", 
    'snap_TX': 'float32', 
    'snap_WI': 'float32' }
price_dtype = {
    "store_id": "category", 
    "item_id": "category", 
    "wm_yr_wk": "int16",
    "sell_price":"float32" }
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
numcols = [f"d_{day}" for day in range(1,29)]
sales_dtype = {numcol:"float32" for numcol in numcols} 
sales_dtype.update({col: "category" for col in catcols if col != "id"})